In [2]:
import pandas as pd
import numpy as np
import json
import fp_growth_py3 as fpg
import matplotlib.pyplot as plt
import pylab as pl
from collections import defaultdict


filepath='test_data/user_following_animation.json'
data=pd.read_json(filepath,lines=True)
data_list = list(data["value"].dropna())
print(len(data_list))

frequent_itemsets = fpg.find_frequent_itemsets(data_list, minimum_support=0.05 * len(data_list), include_support=True)
print(type(frequent_itemsets))  # print type
result = []
for itemset, support in frequent_itemsets:  # 将generator结果存入list
    result.append((itemset, support / len(data_list)))

result_patterns = [i[0] for i in result]
result_support = [i[1] for i in result]
patterns_df = pd.DataFrame({"fluent_patterns": result_patterns, "support": result_support})
patterns = {}
for i in result:
    patterns[frozenset(sorted(i[0]))] = i[1]
print("-------------挖掘频繁项集---------------")
print(patterns_df)


def generate_rules(patterns, min_confidence):
    patterns_group = group_patterns_by_length(patterns)
    raw_rules = defaultdict(set)
    for length, pattern_list in patterns_group.items():
        if length == 1:
            continue
        for pattern, support in pattern_list:
            item_list = list(pattern)
            for window_size in range(1, length):
                for i in range(0, length - window_size):
                    for j in range(i + window_size, length):
                        base_set = frozenset(item_list[i:j])
                        predict_set = frozenset(pattern - base_set)
                        confidence = support / patterns.get(base_set)
                        if confidence > min_confidence:
                            raw_rules[base_set].add((predict_set, confidence))

                        base_set, predict_set = predict_set, base_set
                        confidence = support / patterns.get(base_set)
                        if confidence > min_confidence:
                            raw_rules[base_set].add((predict_set, confidence))
    return raw_rules

def group_patterns_by_length(patterns):
    result = defaultdict(list)
    for pattern, support in patterns.items():
        result[len(pattern)].append((pattern, support))
    return result

def transform(raw_rules):
    result = list()
    for base_set, predict_set_list in raw_rules.items():
        for predict_set, confidence in predict_set_list:
            result.append((base_set, predict_set, confidence))

    return result
raw_rules = generate_rules(patterns, 0.5)
rules = transform(raw_rules)
rules.sort(key=lambda x: x[2], reverse=True)
rules_a = [i[0] for i in rules]
rules_b = [i[1] for i in rules]
confidence = [i[2] for i in rules]
rules_df = pd.DataFrame({"rules_a": rules_a, "rules_b": rules_b, "confidence": confidence})
print(rules_df)

6752
<class 'generator'>
-------------挖掘频繁项集---------------
                fluent_patterns   support
0                      [132112]  0.062352
1                        [6463]  0.091528
2                [102392, 6463]  0.052577
3                  [8892, 6463]  0.051836
4                       [23352]  0.065906
5                       [11712]  0.177873
6               [102392, 11712]  0.095379
7               [139252, 11712]  0.087826
8       [102392, 139252, 11712]  0.053614
9                 [8892, 11712]  0.095231
10        [102392, 8892, 11712]  0.056872
11        [139252, 8892, 11712]  0.059390
12             [5267730, 11712]  0.093898
13     [102392, 5267730, 11712]  0.056280
14     [139252, 5267730, 11712]  0.066055
15       [8892, 5267730, 11712]  0.062056
16              [130412, 11712]  0.067832
17      [139252, 130412, 11712]  0.053466
18     [5267730, 130412, 11712]  0.050652
19             [4312482, 11712]  0.062648
20                [3450, 11712]  0.061908
21              

In [3]:
filepath='./user_following_animation.json'
data=pd.read_json(filepath,lines=True)

In [37]:
vip_data = pd.read_csv("bilibili_crawler_user_info.csv",names = ['id','mid','name','sex','sign','rank','level','jointime','moral','silence','birthday','coins','fans_badge','role','title','desc','vip_type','vip_status'])

In [38]:
vip_data.head()


,id,mid,name,sex,sign,rank,level,jointime,moral,silence,birthday,coins,fans_badge,role,title,desc,vip_type,vip_status
0,1,17515174,试图让群主女装,NaN,NaN,10000.0,5,0.0,0.0,0.0,02-24,0.0,0.0,0.0,NaN,NaN,2.0,1.0
1,2,7596516,酒客小丑,NaN,沉迷于吃狗粮中。商业合作请加QQ2375432747 微博：@酒客小丑,10000.0,6,0.0,0.0,0.0,06-15,0.0,1.0,1.0,bilibili 2018百大UP主、高能联盟成员,NaN,2.0,1.0
2,3,10769575,文史阁,NaN,地势坤，君子以厚德载物。,10000.0,6,0.0,0.0,0.0,01-01,0.0,0.0,1.0,bilibili 知名科普UP主,代表作：中国历代帝王合集,0.0,0.0
3,4,22989278,果冻萌萌的,NaN,林正英以及香港经典鬼片僵尸片24小时播放 房间号：193 欢迎大家观看,10000.0,6,0.0,0.0,0.0,09-17,0.0,1.0,1.0,bilibili 知名UP主,NaN,1.0,0.0
4,5,130050712,初阳手绘石头,NaN,如果搜不到我，说明你是在查找你的好友里搜，当然搜不到了。因为我还不是你的好友呢，你要在添＋好...,10000.0,5,0.0,0.0,0.0,01-01,0.0,1.0,0.0,NaN,NaN,2.0,1.0


In [138]:
print(len(vip_data))
print(len(set(vip_data.mid)))
a=set()
for index,row in vip_data.iterrows():
    if row.mid in a:
        print()

        print(row)
    else:
        a.add(row.mid)
            

51432
51429

id            定期搬运各种科普游戏视频，欢迎关注；想来我们不正规字幕组或唠嗑的小伙伴，请加入交流QQ群：7...
mid                                                       10000
name                                                          6
sex                                                           0
sign                                                          0
rank                                                          0
level                                                     05-05
jointime                                                      0
moral                                                         1
silence                                                       0
birthday                                                    NaN
coins                                                       NaN
fans_badge                                                    1
role                                                          0
title                                                       NaN
desc                       

In [141]:
vip_data[vip_data.mid==10000]

,id,mid,name,sex,sign,rank,level,jointime,moral,silence,birthday,coins,fans_badge,role,title,desc,vip_type,vip_status
2535,NaN,10000,5,0.0,0,0.0,10-16,0.0,0.0,0.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN
21458,定期搬运各种科普游戏视频，欢迎关注；想来我们不正规字幕组或唠嗑的小伙伴，请加入交流QQ群：7...,10000,6,0.0,0,0.0,05-05,0.0,1.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
39014,NaN,10000,5,0.0,0,1.0,NaN,0.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
44169,"最近迷上做神踩点视频""",10000,3,0.0,0,0.0,03-15,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [119]:
vip_data[vip_data.vip_type.isna()]


,id,mid,name,sex,sign,rank,level,jointime,moral,silence,birthday,coins,fans_badge,role,title,desc,vip_type,vip_status
2534,2535,5398071,吾谶,NaN,CP可以逆 本命不能移,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2535,NaN,10000,5,0.0,0,0.0,10-16,0.0,0.0,0.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN
21457,21457,177589181,哔哩哔哩安全应急响应中心,NaN,"><img src=1 onerror=(alert)(2233)>,10000,3,0,0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21458,定期搬运各种科普游戏视频，欢迎关注；想来我们不正规字幕组或唠嗑的小伙伴，请加入交流QQ群：7...,10000,6,0.0,0,0.0,05-05,0.0,1.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
39013,39020,77818,雾香·楼兰,NaN,之前账号：科洛丝琳希停用,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39014,NaN,10000,5,0.0,0,1.0,NaN,0.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
44168,44174,402348056,苍蓝誓约手游,NaN,初次见面！您...就是指挥官大人吗？emmm，跟想象中不太一样呢~总之，请多多指教喽，指挥官...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44169,"最近迷上做神踩点视频""",10000,3,0.0,0,0.0,03-15,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [97]:
vip_add_rules = {0:0,1:1,2:2,3:3,4:4,5:5,6:6}

In [101]:
vip_add_rules[3.0]

4

In [96]:
for index,row in vip_data.head().iterrows():
    print(row.vip_type,' ',type(row.vip_type))
    

2.0   <class 'float'>
2.0   <class 'float'>
0.0   <class 'float'>
1.0   <class 'float'>
2.0   <class 'float'>


In [66]:
data.head()

,db,key,size,ttl,type,value
0,7,330817737,6,-1,set,[132112]
1,7,321510739,9,-1,set,"[6463, 23352]"
2,7,27209511,20,-1,set,"[425, 2580, 5069, 6446, 11712]"
3,7,24774761,12,-1,set,"[5550, 5849, 5852]"
4,7,16686749,259,-1,set,"[53, 710, 835, 844, 1547, 3365, 3398, 3494, 43..."


In [90]:
b=data[data.key=='321510739']

test = pd.DataFrame(columns=data.columns)
for i in range(5):
    test = test.append(b,ignore_index=True)
test

,db,key,size,ttl,type,value
0,7,321510739,9,-1,set,"[6463, 23352]"
1,7,321510739,9,-1,set,"[6463, 23352]"
2,7,321510739,9,-1,set,"[6463, 23352]"
3,7,321510739,9,-1,set,"[6463, 23352]"
4,7,321510739,9,-1,set,"[6463, 23352]"


In [88]:
test.loc[0].db=5
test

,db,key,size,ttl,type,value
0,5,321510739,9,-1,set,"[6463, 23352]"
1,7,321510739,9,-1,set,"[6463, 23352]"
2,7,321510739,9,-1,set,"[6463, 23352]"
3,7,321510739,9,-1,set,"[6463, 23352]"
4,7,321510739,9,-1,set,"[6463, 23352]"


In [58]:

df = pd.DataFrame({'BoolCol': [1, 2, 3, 3, 4],'attr': [22, 33, 22, 44, 66]},       index=[10,20,30,40,50])

In [131]:
df

,BoolCol,attr
10,1,22
20,2,33
30,3,22
40,3,44
50,4,66


In [62]:
a=df[(df.BoolCol==3)&(df.attr==22)].index.tolist()

In [134]:
print(df.iloc[0])

BoolCol     1
attr       22
Name: 10, dtype: int64


输入：配置文件  排名、等级、是否认证、是否VIP
输出：新的Data